In [2]:
import pandas as pd
import os
import numpy as np
import concurrent.futures
from geopy.distance import geodesic

## Here, we consolidate all the data from June 2024 into a single CSV file.

In [3]:
# The CSV files to merge
csv_files = [
    'csvFolders/202406/202406-citibike-tripdata_1.csv', 
    'csvFolders/202406/202406-citibike-tripdata_2.csv', 
    'csvFolders/202406/202406-citibike-tripdata_3.csv',
    'csvFolders/202406/202406-citibike-tripdata_4.csv',
    'csvFolders/202406/202406-citibike-tripdata_5.csv'
]

In [4]:
# Create an empty dataframe to store all data
June2024_df = pd.DataFrame()

# Loop through the list of files and concatenate all data to the dataframe
for csv in csv_files:
    data = pd.read_csv(csv)  # Use the 'csv' variable to read each file
    June2024_df = pd.concat([June2024_df, data], axis=0)

/var/folders/10/nrrfd9q128d9x2q1vyyb65wh0000gn/T/ipykernel_21190/3955821363.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv)  # Use the 'csv' variable to read each file
/var/folders/10/nrrfd9q128d9x2q1vyyb65wh0000gn/T/ipykernel_21190/3955821363.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv)  # Use the 'csv' variable to read each file
/var/folders/10/nrrfd9q128d9x2q1vyyb65wh0000gn/T/ipykernel_21190/3955821363.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv)  # Use the 'csv' variable to read each file
/var/folders/10/nrrfd9q128d9x2q1vyyb65wh0000gn/T/ipykernel_21190/3955821363.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv)  # Use the 'csv' variable to rea

In [17]:
June2024_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A38B171B6C31691A,electric_bike,2024-06-12 17:47:23.952,2024-06-12 17:50:27.312,E 89 St & 3 Ave,7265.10,1 Ave & E 94 St,7286.05,40.780499,-73.952185,40.781721,-73.945940,member
1,77E13C9D6559EA56,electric_bike,2024-06-04 16:48:42.876,2024-06-04 17:01:46.672,Pier 40 - Hudson River Park,5696.03,W 25 St & 9 Ave,6339.06,40.728060,-74.011184,40.747833,-74.000572,member
2,EA6418AD826D19D9,electric_bike,2024-06-01 13:36:28.502,2024-06-01 13:52:55.912,W 44 St & 5 Ave,6551.02,E 75 St & 3 Ave,6991.12,40.754621,-73.980641,40.771129,-73.957723,member
3,121CF2548FB65A29,classic_bike,2024-06-01 03:12:09.636,2024-06-01 03:16:29.531,Graham Ave & Withers St,5403.04,Meserole Ave & Manhattan Ave,5666.04,40.716981,-73.944859,40.727086,-73.952991,member
4,E6BE7FD2809649D6,electric_bike,2024-06-08 13:14:31.197,2024-06-08 13:24:09.858,E 9 St & Ave C,5616.01,Cleveland Pl & Spring St,5492.05,40.725241,-73.977735,40.722104,-73.997249,member


In [18]:
# Remove specific columns from the dataframe (correctly selecting the columns)
June2024_df_cleaned = June2024_df[['rideable_type', 'started_at', 'ended_at', 'start_station_name', 
                                   'start_station_id', 'end_station_name', 
                                   'end_station_id', 'start_lat', 
                                   'start_lng', 'end_lat', 'end_lng', 'member_casual']]

June2024_df_cleaned.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,electric_bike,2024-06-12 17:47:23.952,2024-06-12 17:50:27.312,E 89 St & 3 Ave,7265.10,1 Ave & E 94 St,7286.05,40.780499,-73.952185,40.781721,-73.945940,member
1,electric_bike,2024-06-04 16:48:42.876,2024-06-04 17:01:46.672,Pier 40 - Hudson River Park,5696.03,W 25 St & 9 Ave,6339.06,40.728060,-74.011184,40.747833,-74.000572,member
2,electric_bike,2024-06-01 13:36:28.502,2024-06-01 13:52:55.912,W 44 St & 5 Ave,6551.02,E 75 St & 3 Ave,6991.12,40.754621,-73.980641,40.771129,-73.957723,member
3,classic_bike,2024-06-01 03:12:09.636,2024-06-01 03:16:29.531,Graham Ave & Withers St,5403.04,Meserole Ave & Manhattan Ave,5666.04,40.716981,-73.944859,40.727086,-73.952991,member
4,electric_bike,2024-06-08 13:14:31.197,2024-06-08 13:24:09.858,E 9 St & Ave C,5616.01,Cleveland Pl & Spring St,5492.05,40.725241,-73.977735,40.722104,-73.997249,member


In [7]:
# Save the combined dataframe to a new CSV file
June2024_df_cleaned.to_csv('Resource/202406_citibike_tripdata_all.csv', index=False)
print("CSV files have been successfully merged and saved!")

CSV files have been successfully merged and saved!


## In this section, we clean the data from June 2013 and save the processed version as a CSV file.

In [ ]:
# Read the CSV file
csv_file = 'csvFolders/201306-citibike-tripdata.csv'
June2013_df = pd.read_csv(csv_file)

June2013_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1
2,2059,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,19599,Customer,NaN,0
3,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1
4,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1


In [ ]:
# Remove specific columns from the dataframe (correctly selecting the columns)
June2013_df_cleaned = June2013_df[['starttime', 'stoptime', 'start station id', 'start station name', 
                                   'start station latitude', 'start station longitude', 
                                   'end station id', 'end station name', 
                                   'end station latitude', 'end station longitude', 'usertype']]

June2013_df_cleaned.head()


,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,usertype
0,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,Subscriber
1,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,Subscriber
2,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,Customer
3,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,Subscriber
4,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,Subscriber


In [ ]:
# Rename the column from 'usertype' to 'member_casual'
June2013_df_cleaned = June2013_df_cleaned.rename(columns={'usertype': 'member_casual'})

# Replace values in the 'member_casual' column
June2013_df_cleaned['member_casual'] = June2013_df_cleaned['member_casual'].replace({'Subscriber': 'member', 'Customer': 'casual'})

June2013_df_cleaned.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,member_casual
0,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,member
1,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,member
2,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,casual
3,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,member
4,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,member


In [ ]:
# Save the combined dataframe to a new CSV file
June2013_df_cleaned.to_csv('Resource/201306_citibike_tripdata_cleaned.csv', index=False)
print("CSV files have been successfully merged and saved!")

CSV files have been successfully merged and saved!
